In [ ]:
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2

#my resolution
#1366 x 768
winHeight = int(768 * 0.9)
winWidth = int(1366 * 0.9)

cv2.namedWindow('window', cv2.WINDOW_NORMAL)
cv2.resizeWindow('window', winWidth,winHeight)
#cap = cv2.VideoCapture(0)
vs = VideoStream(src=0).start()
time.sleep(2.0)


# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

while True:
    #ret , frame = cap.read()
    frame = vs.read()
    #scale down
    height, width = frame.shape[:2]
    frame = imutils.resize(frame, int(width* 0.6), int(height * 0.6))


    # detect people in the image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    (rects, weights) = hog.detectMultiScale(gray, winStride=(2, 2), padding=(2, 2), scale=1.05, hitThreshold = 0.5)

    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.7)
    

    for i,(xA, yA, xB, yB) in enumerate(pick):
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
        cv2.rectangle(frame, (xA+2, yA-8), (xA+90, yA), (0, 255, 0), 8) #BASELINE
        cv2.putText(frame, 'Confidence : %.1f' % (weights[i]), (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 0), 1)

    cv2.putText(frame, 'Person : %d' % (len(rects)) , (0,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255))   
    cv2.imshow("window", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
vs.stop()
cv2.destroyAllWindows()